# Customer Service Pipeline Exercise

**Sequential Flow**: Classify Agent → Process Agent → Respond Agent

This exercise demonstrates:
- Three specialized agents for support workflow
- Ticket classification and routing
- Context gathering and personalized responses

In [1]:
from strands import Agent, tool
from strands.models import BedrockModel
import json

## Define Support Tools

In [2]:
@tool
def get_customer_history(customer_id: str) -> str:
    """Retrieve customer account history and previous interactions"""
    history = {
        "customer_id": customer_id,
        "account_status": "Premium",
        "member_since": "2023-05-15",
        "total_orders": 24,
        "lifetime_value": 3450,
        "previous_tickets": [
            {"date": "2025-11-20", "issue": "Shipping delay", "status": "Resolved"},
            {"date": "2025-09-10", "issue": "Product question", "status": "Resolved"}
        ]
    }
    return json.dumps(history)

@tool
def get_order_details(order_id: str) -> str:
    """Retrieve specific order information"""
    order = {
        "order_id": order_id,
        "date": "2025-12-15",
        "status": "Shipped",
        "tracking": "1Z999AA10123456784",
        "items": ["Wireless Headphones", "USB-C Cable"],
        "total": 149.99,
        "estimated_delivery": "2025-12-20"
    }
    return json.dumps(order)

@tool
def search_knowledge_base(query: str) -> str:
    """Search internal knowledge base for solutions"""
    results = {
        "query": query,
        "articles": [
            {"title": "How to track your order", "url": "kb/track-order"},
            {"title": "Return policy guide", "url": "kb/returns"},
            {"title": "Shipping timeframes", "url": "kb/shipping"}
        ]
    }
    return json.dumps(results)

## Step 1: Create Classify Agent

**Purpose**: Categorize support tickets by type, priority, and urgency

In [4]:
classify_agent = Agent(
    name="classify_agent",
    system_prompt="""You are a support ticket classification specialist.
    
    Your role:
    - Categorize tickets into: Billing, Shipping, Product, Technical, Account, Other
    - Assign priority: Low, Medium, High, Urgent
    - Identify sentiment: Positive, Neutral, Negative, Angry
    - Extract key entities: customer_id, order_id, product_name
    - Flag if escalation needed
    
    Output format (JSON):
    {
      "category": "[category]",
      "priority": "[priority]",
      "sentiment": "[sentiment]",
      "entities": {"customer_id": "...", "order_id": "..."},
      "escalate": true/false,
      "summary": "[one sentence summary]"
    }""",
    model=BedrockModel(model_id="us.amazon.nova-micro-v1:0")
)

## Step 2: Create Process Agent

**Purpose**: Gather relevant context, history, and supporting information

In [5]:
process_agent = Agent(
    name="process_agent",
    system_prompt="""You are a support context specialist.
    
    Your role:
    - Use tools to gather customer history, order details, and knowledge base articles
    - Compile all relevant context for the ticket
    - Identify potential solutions from knowledge base
    - Note any special customer considerations (VIP, previous issues, etc.)
    
    Output format:
    ## Customer Context
    [Customer history and status]
    
    ## Issue Context
    [Order/product details if applicable]
    
    ## Relevant Solutions
    [Knowledge base articles and recommendations]
    
    ## Special Notes
    [Any important considerations]""",
    model=BedrockModel(model_id="us.amazon.nova-micro-v1:0"),
    tools=[get_customer_history, get_order_details, search_knowledge_base]
)

## Step 3: Create Respond Agent

**Purpose**: Draft personalized, empathetic customer response

In [6]:
respond_agent = Agent(
    name="respond_agent",
    system_prompt="""You are a customer service response specialist.
    
    Your role:
    - Draft personalized, empathetic responses
    - Address the customer by name if available
    - Acknowledge their concern with empathy
    - Provide clear, actionable solutions
    - Include relevant links or next steps
    - Match tone to customer sentiment (more empathy for negative sentiment)
    - Keep response concise but complete (150-250 words)
    
    Response structure:
    1. Greeting with personalization
    2. Empathetic acknowledgment
    3. Clear solution or explanation
    4. Next steps or additional help
    5. Professional closing""",
    model=BedrockModel(model_id="us.amazon.nova-micro-v1:0")
)

## Step 4: Implement Sequential Workflow

In [7]:
def customer_service_pipeline(ticket: str) -> dict:
    """
    Execute sequential customer service pipeline.
    
    Args:
        ticket: Customer support ticket text
    
    Returns:
        Dictionary with outputs from each stage
    """
    print("=" * 60)
    print("STAGE 1: TICKET CLASSIFICATION")
    print("=" * 60)
    
    # Stage 1: Classify the ticket
    classification = classify_agent(f"Classify this support ticket:\n\n{ticket}")
    print(classification)
    
    print("\n" + "=" * 60)
    print("STAGE 2: CONTEXT PROCESSING")
    print("=" * 60)
    
    # Stage 2: Gather context
    process_prompt = f"""Gather all relevant context for this ticket:
    
TICKET:
{ticket}

CLASSIFICATION:
{classification}
"""
    context = process_agent(process_prompt)
    print(context)
    
    print("\n" + "=" * 60)
    print("STAGE 3: RESPONSE GENERATION")
    print("=" * 60)
    
    # Stage 3: Generate response
    response_prompt = f"""Draft a customer response for this ticket:
    
ORIGINAL TICKET:
{ticket}

CLASSIFICATION:
{classification}

CONTEXT:
{context}
"""
    response = respond_agent(response_prompt)
    print(response)
    
    return {
        "classification": str(classification),
        "context": str(context),
        "response": str(response)
    }

## Step 5: Test with Realistic Tickets

In [8]:
# Test Case 1: Shipping inquiry
ticket1 = """
Subject: Where is my order?
From: sarah.johnson@email.com
Customer ID: CUST-12345
Order ID: ORD-98765

Hi, I ordered wireless headphones on December 15th and haven't received them yet. 
The tracking shows it's still in transit but it was supposed to arrive by December 20th. 
Can you help me figure out what's going on?
"""

result = customer_service_pipeline(ticket1)

STAGE 1: TICKET CLASSIFICATION
{
  "category": "Shipping",
  "priority": "Medium",
  "sentiment": "Neutral",
  "entities": {
    "customer_id": "CUST-12345",
    "order_id": "ORD-98765"
  },
  "escalate": false,
  "summary": "A customer is inquiring about the status and delay of their in-transit order of wireless headphones."
}{
  "category": "Shipping",
  "priority": "Medium",
  "sentiment": "Neutral",
  "entities": {
    "customer_id": "CUST-12345",
    "order_id": "ORD-98765"
  },
  "escalate": false,
  "summary": "A customer is inquiring about the status and delay of their in-transit order of wireless headphones."
}


STAGE 2: CONTEXT PROCESSING
<thinking>To gather all relevant context for this ticket, I need to use the provided customer ID and order ID to retrieve the customer's history, order details, and any relevant knowledge base articles. Here is the plan:

1. Retrieve the customer's history using the customer ID.
2. Retrieve the specific order details using the order ID.
3. 

In [9]:
# Test Case 2: Frustrated customer
ticket2 = """
Subject: THIRD TIME contacting support!!!
From: angry.customer@email.com
Customer ID: CUST-67890

This is ridiculous! I've contacted support twice already about my account being charged 
twice for the same order. Nobody has fixed this yet and it's been a week! 
I want a refund immediately or I'm canceling my account and leaving a bad review.
"""

result = customer_service_pipeline(ticket2)

STAGE 1: TICKET CLASSIFICATION
{
  "category": "Billing",
  "priority": "Urgent",
  "sentiment": "Angry",
  "entities": {
    "customer_id": "CUST-67890"
  },
  "escalate": true,
  "summary": "An angry customer demands a refund and threatens to leave a bad review due to being charged twice for the same order."
}{
  "category": "Billing",
  "priority": "Urgent",
  "sentiment": "Angry",
  "entities": {
    "customer_id": "CUST-67890"
  },
  "escalate": true,
  "summary": "An angry customer demands a refund and threatens to leave a bad review due to being charged twice for the same order."
}


STAGE 2: CONTEXT PROCESSING
<thinking>To gather all relevant context for this ticket, I need to use the provided customer ID to retrieve the customer's history. Given the urgency and escalation flag, it's important to quickly provide the customer with all necessary information and a clear path to resolution. Here is the plan:

1. Retrieve the customer's history using the customer ID.
2. Identify the

In [10]:
# Test Case 3: Product question
ticket3 = """
Subject: Question about product compatibility
From: tech.user@email.com
Customer ID: CUST-11111

Hello! I'm interested in purchasing the USB-C cable but I want to make sure 
it's compatible with my MacBook Pro 2024. Can you confirm this works with 
the latest models? Also, does it support fast charging?

Thanks!
"""

result = customer_service_pipeline(ticket3)

STAGE 1: TICKET CLASSIFICATION
{
  "category": "Product",
  "priority": "Low",
  "sentiment": "Positive",
  "entities": {
    "customer_id": "CUST-11111",
    "product_name": "USB-C cable"
  },
  "escalate": false,
  "summary": "A customer is inquiring about the compatibility and fast charging support of a USB-C cable with their MacBook Pro 2024."
}{
  "category": "Product",
  "priority": "Low",
  "sentiment": "Positive",
  "entities": {
    "customer_id": "CUST-11111",
    "product_name": "USB-C cable"
  },
  "escalate": false,
  "summary": "A customer is inquiring about the compatibility and fast charging support of a USB-C cable with their MacBook Pro 2024."
}


STAGE 2: CONTEXT PROCESSING
## Customer Context
- **Customer ID**: CUST-11111
- **Account Status**: Not specified in the context
- **Member Since**: Not provided
- **Total Orders**: Not provided
- **Lifetime Value**: Not provided
- **Previous Tickets**: Not provided

## Issue Context
- **Category**: Product
- **Priority**: L

## Step 6: Inspect Pipeline Stages

In [11]:
# Access individual stage outputs
print("=== CLASSIFICATION ===")
print(result['classification'][:300] + "...\n")

print("=== CONTEXT ===")
print(result['context'][:300] + "...\n")

print("=== RESPONSE ===")
print(result['response'])

=== CLASSIFICATION ===
{
  "category": "Product",
  "priority": "Low",
  "sentiment": "Positive",
  "entities": {
    "customer_id": "CUST-11111",
    "product_name": "USB-C cable"
  },
  "escalate": false,
  "summary": "A customer is inquiring about the compatibility and fast charging support of a USB-C cable with their ...

=== CONTEXT ===
## Customer Context
- **Customer ID**: CUST-11111
- **Account Status**: Not specified in the context
- **Member Since**: Not provided
- **Total Orders**: Not provided
- **Lifetime Value**: Not provided
- **Previous Tickets**: Not provided

## Issue Context
- **Category**: Product
- **Priority**: Low...

=== RESPONSE ===
**Subject:** Confirming Compatibility and Fast Charging Support for Your USB-C Cable

**Dear Tech User,**

Thank you for reaching out with your inquiry about our USB-C cable. We're pleased to hear about your interest and are here to help ensure you make an informed purchase decision.

**Compatibility:**
The USB-C cable we offer is d

## Exercise Variations

Try modifying the pipeline:

1. **Add Sentiment Analysis Agent** for deeper emotion detection
2. **Add Auto-Resolution Agent** for simple tickets
3. **Add Quality Check Agent** to review responses before sending
4. **Implement routing logic** based on classification (escalate urgent tickets)
5. **Add feedback loop** to learn from customer satisfaction ratings

## Key Learnings

✅ **Sequential Flow**: Classify → Process → Respond  
✅ **Tool Integration**: Process agent uses multiple context tools  
✅ **Context Passing**: Full ticket context flows through pipeline  
✅ **Personalization**: Response adapts to sentiment and customer history  
✅ **Structured Classification**: JSON output enables routing logic  